# CSVS to Hepfile

Imagine we have a database like structure with multiple csvs all connected by a common ID. See the household exmaple here: https://hepfile.readthedocs.io/en/latest/fundamentals.html#a-toy-example 

In [1]:
import os, glob
import pandas as pd
import numpy as np
import awkward as ak
import hepfile as hf

In [2]:
datapath = os.path.join('/', 'home', 'nfranz', 'research', 'hepfile', 'examples', '*.csv')
files = glob.glob(datapath)
common_key = 'Household ID'
group_names = ['Residences', 'People', 'Vehicles']

if group_names is None:
    group_names = [os.path.split(file) for file in files]

for_ak = {}
for f, group_name in zip(files, group_names):
    
    csv = pd.read_csv(f)
    
    groups = csv.groupby(common_key)
    split_groups = []
    for item in groups.groups:
        split_groups.append(groups.get_group(item))
    
    subdict = {}
    for grouping in split_groups:
        for colname in grouping.columns:
            if colname in subdict.keys():
                subdict[colname].append(list(grouping[colname].values))
            else:
                subdict[colname] = [list(grouping[colname].values)]
                
    for key in subdict.keys():
        subdict[key] = ak.Array(subdict[key])
        
    
    for_ak[group_name] = subdict
for_ak

{'Residences': {'Household ID': <Array [[0], [1], [2], [3]] type='4 * var * int64'>,
  'House/apartment/condo': <Array [['House'], ['Apartment'], ..., ['House']] type='4 * var * string'>,
  '# of bedrooms': <Array [[4], [2], [2], [6]] type='4 * var * int64'>,
  '# of bathrooms': <Array [[2.5], [2], [1], [4.5]] type='4 * var * float64'>,
  'Square footage': <Array [[1500], [1200], [1000], [4500]] type='4 * var * int64'>,
  'Year built': <Array [[1955], [2002], [2014], [1998]] type='4 * var * int64'>,
  'Estimate': <Array [[250000], [1400], [325000], [500000]] type='4 * var * int64'>},
 'People': {'Household ID': <Array [[0, 0, 0, 0], [...], ..., [3, 3, 3, 3, 3, 3, 3]] type='4 * var * int64'>,
  'First name': <Array [['blah', 'blah', 'blah', 'blah'], ..., [...]] type='4 * var * string'>,
  'Last name': <Array [['blah', 'blah', 'blah', 'blah'], ..., [...]] type='4 * var * string'>,
  'Gender ID': <Array [['M', 'F', 'NB', 'F'], ..., ['M', ..., 'F']] type='4 * var * string'>,
  'Age': <Arra

In [3]:
hf.awkward_tools.awkward_to_hepfile(for_ak, 'testing_csv_translation.h5')

Adding group Residences
Adding a counter for Residences as nResidences
----------------------------------------------------
Slashes / are not allowed in dataset names
Replacing / with - in dataset name House/apartment/condo
The new name will be House-apartment-condo
----------------------------------------------------
Adding dataset Household ID to the dictionary under group Residences.
Adding dataset House-apartment-condo to the dictionary under group Residences.
Adding dataset # of bedrooms to the dictionary under group Residences.
Adding dataset # of bathrooms to the dictionary under group Residences.
Adding dataset Square footage to the dictionary under group Residences.
Adding dataset Year built to the dictionary under group Residences.
Adding dataset Estimate to the dictionary under group Residences.
Adding group People
Adding a counter for People as nPeople
----------------------------------------------------
Slashes / are not allowed in dataset names
Replacing / with - in datas

/home/nfranz/anaconda3/lib/python3.9/site-packages/hepfile/write.py:560: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if group == "_SINGLETONS_GROUP_" and dataset is not "COUNTER":
/home/nfranz/anaconda3/lib/python3.9/site-packages/hepfile/write.py:560: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if group == "_SINGLETONS_GROUP_" and dataset is not "COUNTER":


ValueError: could not convert string to float: 'House'

This error occurred while calling

    numpy.asarray(
        <Array ['House', 'Apartment', 'Condo', 'House'] type='4 * string'>
        dtype[float64]-instance
    )